# Ollama + OpenAI + Python

## 1. மாடல் பெயரை குறிப்பிடவும்

நீங்கள் "phi3:mini" என்ற மாடலின் பதிலாக வேறு ஒரு மாடலை ஏற்றியிருந்தால், கீழே உள்ள செலில் உள்ள மதிப்பை மாற்றவும்.
அந்த மாறி நோட்புக் முழுவதும் உள்ள கோடுகளில் பயன்படுத்தப்படும்.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AI கிளையன்டை அமைத்தல்

பொதுவாக OpenAI கிளையன்ட் OpenAI.com அல்லது Azure OpenAI உடன் பெரிய மொழி மாதிரிகளுடன் தொடர்பு கொள்ள பயன்படுத்தப்படுகிறது.
ஆனால், இது Ollama உடனும் பயன்படுத்தப்படலாம், ஏனெனில் Ollama "http://localhost:11434/v1" என்ற இடத்தில் OpenAI-இன் இணக்கமான endpoint-ஐ வழங்குகிறது.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. ஒரு உரையாடல் முடிவை உருவாக்கு

இப்போது OpenAI SDK ஐப் பயன்படுத்தி ஒரு உரையாடலுக்கு பதிலை உருவாக்க முடியும். இந்த கோரிக்கை பூனைகள் பற்றிய ஒரு ஹைகுவை உருவாக்க வேண்டும்:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. ப்ராம்ட் இன்ஜினியரிங்

மொழி மாதிரிக்கு அனுப்பப்படும் முதல் செய்தியை "சிஸ்டம் செய்தி" அல்லது "சிஸ்டம் ப்ராம்ட்" என்று அழைக்கின்றனர், இது மாதிரிக்கான மொத்த வழிமுறைகளை நிர்ணயிக்கிறது.
நீங்கள் உங்கள் சொந்த தனிப்பயன் சிஸ்டம் ப்ராம்ட்டை அளித்து, மொழி மாதிரியை வேறொரு முறையில் வெளியீடு உருவாக்க வழிநடத்தலாம்.
கீழே உள்ள `SYSTEM_MESSAGE` ஐ உங்கள் பிடித்த பிரபலமான திரைப்பட/தொலைக்காட்சி பாத்திரம் போல பதில் அளிக்க மாற்றவும், அல்லது பிற சிஸ்டம் ப்ராம்ட்களுக்கு ஊக்கத்தை [அற்புதமான ChatGPT தூண்டுதல்கள்](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) இலிருந்து பெறவும்.

ஒரு முறை நீங்கள் சிஸ்டம் செய்தியை தனிப்பயனாக்கிய பிறகு, முதல் பயனர் கேள்வியை `USER_MESSAGE` இல் வழங்கவும்.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. சில-ஷாட் எடுத்துக்காட்டுகள்

மொழி மாதிரியை வழிநடத்துவதற்கான மற்றொரு வழி "few shots" வழங்குவது, இது எப்படி பதிலளிக்க வேண்டும் என்பதை காட்டும் கேள்வி/பதில் எடுத்துக்காட்டுகளின் தொடர்.

கீழே உள்ள எடுத்துக்காட்டு ஒரு மொழி மாதிரியை ஒரு கற்பித்தல் உதவியாளர் (teaching assistant) போல நடிக்கச் செய்வதற்காக TA வழங்கக்கூடிய சில கேள்வி/பதில் எடுத்துக்காட்டுகளை வழங்குகிறது, மற்றும் பின்னர் ஒரு மாணவர் கேட்கக்கூடிய கேள்வியொன்றை கொண்டு மாதிரியை தூண்டும்.

முதலில் இதைக் முயற்சி செய்யவும், பின்னர் புதிய சூழ்நிலைக்கு `SYSTEM_MESSAGE`, `EXAMPLES`, மற்றும் `USER_MESSAGE` ஐ மாற்றுங்கள்.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. மீட்பு விரிவூட்டிய உருவாக்கம்

RAG (Retrieval Augmented Generation) என்பது ஒரு மொழி மாதிரியை குறிப்பிட்ட துறையில் கேள்விகளுக்கு துல்லியமாக பதிலளிக்க செய்யும் ஒரு தொழில்நுட்பமாகும்; முதலில் அறிவுத்தளத்திலிருந்து தொடர்புடைய தகவல்களை மீட்டெடுத்து, பின்னர் அந்த தகவல்களின் அடிப்படையில் பதிலை உருவாக்குகிறது.

நாம் ஹைபிரிட் கார்கள் பற்றிய தரவுகளை கொண்ட ஒரு உள்ளூர் CSV கோப்பினை வழங்கியுள்ளோம். கீழே உள்ள குறியீடு CSV கோப்பை படிக்கிறது, பயனரின் கேள்விக்கு பொருத்தமான பொருட்களைத் தேடுகிறது, மற்றும் கண்டறிந்த தகவல்களின் அடிப்படையில் பதிலை உருவாக்குகிறது. இது முந்தைய எடுத்துக்காட்டுகளில் ஒப்பிடுகையில் அதிக நேரம் எடுத்துக்கொள்ளலாம், ஏனெனில் இது மாதிரிக்கு அதிகமான தரவுகளை அனுப்புகிறது. பதில் இன்னும் தரவு மூலம் ஆதாரபூரணமில்லை என்று நீங்கள் கவனித்தால், நீங்கள் system engineering முயற்சி செய்யலாம் அல்லது வேறொரு மாதிரியை முயற்சிக்கலாம். பொதுவாக, RAG பெரிய மாதிரிகளோ அல்லது SLM-களின் நுணுக்கமாக பயிற்சி செய்யப்பட்ட பதிப்புகளோடு மிகவும் பயனுள்ளதாக இருக்கும்.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டது. நாங்கள் துல்லியத்திற்காக முயலினாலும், தானாக செய்யப்பட்ட மொழிபெயர்ப்புகள் பிழைகள் அல்லது தவறுகளை கொண்டிருக்கலாம் என்பதை தயவுசெய்து நினைவில் கொள்க. மூல ஆவணம் அதன் இயல்பு (அசல்) மொழியில் அதிகாரப்பூர்வ ஆதாரமாகக் கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்குகிறோம். இந்த மொழிபெயர்ப்பு காரணமாக ஏற்படக்கூடிய எந்தவொரு தவறான புரிதலிற்கோ அல்லது தவறான விளக்கங்களிற்கோ நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
